### 네이버 기사 카테고리 분류

##### 1. Import Packages

In [1]:
import pickle

In [2]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

import pandas as pd
import pickle

##### 2. Load Dataset
- Category
    - 0 - 정치
    - 1 - 경제
    - 2 - 사회
    - 3 - 생활/문화
    - 4 - 세계
    - 5 - IT/과학

In [3]:
article_df = pd.read_pickle("../datas/article_2016_06_01.plk")
print(len(article_df))
article_df.tail()

2327


newsid  oid newspaper                                          title  \
2322  2976863   20      동아일보              中, 원자재서 첨단기술로 ‘사냥 타깃’ 이동… 한국은 제자리   
2323  2976847   20      동아일보                       "왓슨 한국어 배우는 중… 연내 데뷔 가능"   
2324  3179706   23      조선일보  [어수웅의 르네상스人] 종이 사전을 삼킨 남자, &#39;웹 사전&#39;을 낳다   
2325  2976797   20      동아일보                     中, M&A로 특허 포식… 한국 미래산업 삼킨다   
2326  2620983   25      중앙일보                      구글·애플 대항마…토종 앱 장터 원스토어 떴다   

                                                   link  comment  likeit  \
2322  http://news.naver.com/main/read.nhn?mode=LSD&m...        5       4   
2323  http://news.naver.com/main/read.nhn?mode=LSD&m...        1       4   
2324  http://news.naver.com/main/read.nhn?mode=LSD&m...        0       2   
2325  http://news.naver.com/main/read.nhn?mode=LSD&m...       13       9   
2326  http://news.naver.com/main/read.nhn?mode=LSD&m...      123       9   

                                                content        date  category  
2322  [동아일보][미래산업 먹어치우는 중국]슈퍼 포식자 중국美 로봇업체 인수… 獨기업 눈...  2016-06-01         5  
2323  [동아일보][2016 동아 국제금융 포럼]‘AI 전문가’ 메네제스 IBM 부사장“A...  2016-06-01         5  
2324  [정철 카카오 웹 사전 기획자] 책 '검색, 사전을 삼키다' 펴내… 연세대에서 사전...  2016-06-01         5  
2325  [동아일보]中, 로봇-IoT-바이오 등 M&A; 규모… 올해 5월에 이미 작년 기록...  2016-06-01         5  
2326  이통 3사, 네이버 손잡고 시장 40% 점유 목표| 양강 구도에 도전장…앱 생태계 ...  2016-06-01         5

##### 3. Split Tratin, Test

In [4]:
X_train, X_test, y_train, y_test = train_test_split(article_df.content, article_df.category, test_size=0.1, random_state=1)
len(X_train), len(X_test), len(y_train), len(y_test)

(2094, 233, 2094, 233)

##### 4. Make Model

In [5]:
clf = Pipeline([
    ('vect', TfidfVectorizer()), 
    ('clf', MultinomialNB(alpha=0.01)),
])

In [6]:
%%time
model = clf.fit(X_train, y_train)

CPU times: user 836 ms, sys: 34.4 ms, total: 870 ms
Wall time: 834 ms


##### 5. Confusion Matrix, Classfication Report

In [7]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

y_pred = model.predict(X_test)

In [8]:
confusion_matrix(y_test, y_pred)

array([[41,  0,  3,  1,  1,  0],
       [ 1, 33,  0,  5,  1,  1],
       [ 1,  2, 77,  2,  1,  0],
       [ 0,  0,  2, 18,  0,  1],
       [ 1,  1,  2,  0, 25,  0],
       [ 0,  1,  1,  0,  0, 11]])

In [9]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.93      0.89      0.91        46
           1       0.89      0.80      0.85        41
           2       0.91      0.93      0.92        83
           3       0.69      0.86      0.77        21
           4       0.89      0.86      0.88        29
           5       0.85      0.85      0.85        13

   micro avg       0.88      0.88      0.88       233
   macro avg       0.86      0.86      0.86       233
weighted avg       0.88      0.88      0.88       233



##### 6. Prediction

In [10]:
X_test = X_test.reset_index(drop=True)

In [11]:
X_test[0][:100], X_test[1][:100], X_test[2][:100]

('인천 송도 삼성바이오로직스 현장을 가다“삼성이 처음 핸드폰 사업을 시작할 때 망치로 핸드폰을 깨부수는 등 온갖 실험을 했던 동영상도 보여주며 ‘이렇게 시작해 휴대폰 사업도 성공했다',
 '[구의역 사고를 보며] 청년을 위한 나라는 없다\xa0[ 주동식 지역평등시민연대 대표]\xa0요즘 사람의 실제 나이는 전통적인 개념의 숫자에 0.8을 곱해서 계산해야 한다고 한다. 50세 중',
 ' [한겨레] 구의역·남양주 사고에 입법대책 분주더민주, 직접 고용법 제정안 발의국민의당, 청년고용할당제 확대정의당도 ‘기업살인법’ 발의키로정부·여당 반대로 통과 어려울듯새누리, 야')

In [12]:
result = model.predict([ X_test[0], X_test[1], X_test[2] ])
result

array([1, 2, 0])

In [13]:
classification_dict = {
    0:"정치",
    1:"경제",
    2:"사회",
    3:"생활/문화",
    4:"세계",
    5:"IT/과학",
}
for idx, category in enumerate(result):
    print(classification_dict[category], "-",X_test[idx][:80])

경제 - 인천 송도 삼성바이오로직스 현장을 가다“삼성이 처음 핸드폰 사업을 시작할 때 망치로 핸드폰을 깨부수는 등 온갖 실험을 했던 동영상도 보여주며 ‘
사회 - [구의역 사고를 보며] 청년을 위한 나라는 없다 [ 주동식 지역평등시민연대 대표] 요즘 사람의 실제 나이는 전통적인 개념의 숫자에 0.8을 곱해
정치 -  [한겨레] 구의역·남양주 사고에 입법대책 분주더민주, 직접 고용법 제정안 발의국민의당, 청년고용할당제 확대정의당도 ‘기업살인법’ 발의키로정부·


##### 7. Save & Load Model

In [14]:
pickle.dump(model, open("classification.plk", "wb"))

In [15]:
load_model = pickle.load(open("classification.plk", "rb"))

In [16]:
test_str1 = "네이버와 카카오 드론 기술 발전에 주력"
test_str2 = "요즘에 환율과 주가가 예측 불허"
print( classification_dict[ load_model.predict([test_str1])[0] ], "-", test_str1 )
print( classification_dict[ load_model.predict([test_str2])[0] ], "-", test_str2 )

IT/과학 - 네이버와 카카오 드론 기술 발전에 주력
경제 - 요즘에 환율과 주가가 예측 불허


In [17]:
# 카테고리별 확률 데이터로 출력
proba = load_model.predict_proba([test_str1])[0]
proba

array([0.01620144, 0.043785  , 0.01565891, 0.08956624, 0.01146436,
       0.82332404])

In [18]:
category = classification_dict.values()
np.array(list(category))

array(['정치', '경제', '사회', '생활/문화', '세계', 'IT/과학'], dtype='<U5')

In [19]:
df = pd.DataFrame({
    "category": np.array(list(category)),
    "proba": proba,
})

df

category     proba
0       정치  0.016201
1       경제  0.043785
2       사회  0.015659
3    생활/문화  0.089566
4       세계  0.011464
5    IT/과학  0.823324